In [1]:
!pip install flwr "protobuf<6" --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.1/727.1 kB 9.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 65.0 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 13.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pyopenssl 25.3.0 requires cryptography<47,>=45.0.7, but you have cryptography 44.0.3 which is incompatible.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.9.2 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 whi

In [4]:
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"
os.environ["RAY_DISABLE_METRICS"] = "1"


In [5]:
import flwr as fl
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split, Subset

print("Flower version:", fl.__version__)
print("Torch version:", torch.__version__)

DEVICE = torch.device("cpu")
print("Using device:", DEVICE)


Flower version: 1.25.0
Torch version: 2.8.0+cu126
Using device: cpu


In [7]:
DATA_DIR = "/kaggle/input/plantvillage-dataset/color"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

full_dataset = datasets.ImageFolder(DATA_DIR, transform=transform)

# 🔥 LIMIT DATA (CRITICAL)
MAX_SAMPLES = 3000
full_dataset = Subset(full_dataset, range(MAX_SAMPLES))

print("Dataset size:", len(full_dataset))
print("Classes:", len(full_dataset.dataset.classes))


Dataset size: 3000
Classes: 38


In [8]:
NUM_CLIENTS = 3

base_len = len(full_dataset) // NUM_CLIENTS
lengths = [base_len] * NUM_CLIENTS
lengths[-1] += len(full_dataset) - sum(lengths)

client_datasets = random_split(full_dataset, lengths)

print("Client sizes:", [len(ds) for ds in client_datasets])


Client sizes: [1000, 1000, 1000]


In [10]:
def create_model():
    model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT)
    model.classifier[1] = nn.Linear(model.last_channel, len(full_dataset.dataset.classes))
    return model.to(DEVICE)


In [12]:
def train(model, loader):
    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    for images, labels in loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


def test(model, loader):
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total


In [14]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, trainloader, testloader):
        self.model = create_model()
        self.trainloader = trainloader
        self.testloader = testloader

    def get_parameters(self, config):
        return [val.cpu().numpy() for val in self.model.state_dict().values()]

    def set_parameters(self, parameters):
        params_dict = zip(self.model.state_dict().keys(), parameters)
        state_dict = {k: torch.tensor(v) for k, v in params_dict}
        self.model.load_state_dict(state_dict, strict=True)

    def fit(self, parameters, config):
        print(">>> Client training started")
        self.set_parameters(parameters)
        train(self.model, self.trainloader)
        print(">>> Client training finished")
        return self.get_parameters(config), len(self.trainloader.dataset), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        acc = test(self.model, self.testloader)
        return float(acc), len(self.testloader.dataset), {"accuracy": acc}


In [15]:
def client_fn(cid):
    dataset = client_datasets[int(cid)]

    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_ds, test_ds = random_split(dataset, [train_size, test_size])

    trainloader = DataLoader(train_ds, batch_size=16, shuffle=True)
    testloader = DataLoader(test_ds, batch_size=16)

    return FlowerClient(trainloader, testloader)


In [17]:
def weighted_average(metrics):
    total, correct = 0, 0
    for num_examples, m in metrics:
        correct += num_examples * m["accuracy"]
        total += num_examples
    return {"accuracy": correct / total}


In [19]:
NUM_ROUNDS = 3

strategy = fl.server.strategy.FedAvg(
    min_fit_clients=NUM_CLIENTS,
    min_available_clients=NUM_CLIENTS,
    evaluate_metrics_aggregation_fn=weighted_average,
)


In [20]:
history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
    strategy=strategy,
)


2026-01-15 10:07:15.621396: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768471635.878260      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768471635.954124      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768471636.600334      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768471636.600391      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768471636.600395      55 computation_placer.cc:177] computation placer alr

(ClientAppActor pid=215) Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-7ebf99e0.pth


100%|██████████| 13.6M/13.6M [00:00<00:00, 136MB/s]
(ClientAppActor pid=215) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
INFO :      Received initial parameters from one random client
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 3)
(ClientAppActor pid=215) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=215) 
(ClientAppActor pid=215)             This is a deprecated feature. It will be removed
(ClientAppActor pid=215)             entirely in futu

(ClientAppActor pid=215) >>> Client training started


(pid=gcs_server) [2026-01-15 10:08:05,888 E 98 98] (gcs_server) gcs_server.cc:303: Failed to establish connection to the event+metrics exporter agent. Events and metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14
(ClientAppActor pid=214) 
(ClientAppActor pid=214)         
(ClientAppActor pid=214) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=214)             This is a deprecated feature. It will be removed
(ClientAppActor pid=214)             entirely in future versions of Flower.
(ClientAppActor pid=212) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid">}. You can import the `

...

INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      configure_evaluate: strategy sampled 3 clients (out of 3)
(ClientAppActor pid=212) 
(ClientAppActor pid=212)         
(ClientAppActor pid=212) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 3x across cluster]
(ClientAppActor pid=212)             This is a deprecated feature. It will be removed [repeated 3x across cluster]
(ClientAppActor pid=212)             entirely in future versions of Flower. [repeated 3x across cluster]
(ClientAppActor pid=214) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`. [repeated 3x across cluster]
(ClientAppActor pid=215)

(ClientAppActor pid=214) >>> Client training started
(ClientAppActor pid=212) >>> Client training finished
(ClientAppActor pid=212) >>> Client training started
(ClientAppActor pid=214) >>> Client training finished
(ClientAppActor pid=215) >>> Client training started
(ClientAppActor pid=215) >>> Client training finished


INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      configure_evaluate: strategy sampled 3 clients (out of 3)


(ClientAppActor pid=212) >>> Client training finished


(ClientAppActor pid=212) 
(ClientAppActor pid=212)         
(ClientAppActor pid=212) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 3x across cluster]
(ClientAppActor pid=212)             This is a deprecated feature. It will be removed [repeated 3x across cluster]
(ClientAppActor pid=212)             entirely in future versions of Flower. [repeated 3x across cluster]
(ClientAppActor pid=215) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`. [repeated 3x across cluster]
(ClientAppActor pid=215) 
(ClientAppActor pid=215)         
(ClientAppActor pid=214) 
(ClientAppActor pid=214)         
INFO :      aggregate_evaluate: rec

In [23]:
print("\nFederated Accuracy per Round:")

for rnd, acc in history.metrics_distributed["accuracy"]:
    print(f"Round {rnd}: Accuracy = {acc:.4f}")

final_acc = history.metrics_distributed["accuracy"][-1][1]
print("\nFinal Federated Accuracy:", final_acc)


print("Week 4 Federated Accuracy :", round(final_acc, 4))


Federated Accuracy per Round:
Round 1: Accuracy = 0.9983
Round 2: Accuracy = 0.9967
Round 3: Accuracy = 0.9983

Final Federated Accuracy: 0.9983333333333333
Week 4 Federated Accuracy : 0.9983
